In [26]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  
import folium
from collections import OrderedDict
from ipygee import *
import eeconvert
import pandas as pd
import numpy as np
import geopandas as gpd

In [27]:
g = os.getcwd()
path = 'D:\\repos\\CIAT\\IDB_PROJECT'
os.chdir(path)

In [28]:
Points = gpd.read_file("Data_IDB/new/Zona_influencia_DR.shp")
covex = gpd.read_file("Data_IDB/new/covexhull.shp")
# ok = Points[Points['corregido'].str.contains("ok")]
# print(ok)
covex = eeconvert.gdfToFc(covex)
covex = covex.geometry()


In [29]:
L7 = (ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
      .filterBounds(covex)# filter according to drawn boundary
      .filter(ee.Filter.lt('CLOUD_COVER', 35))
      .filterDate('2011-01-01', '2011-01-30')
      .select(["B1","B2","B3","B4","B5"])
      .map(lambda image: image.clipToBoundsAndScale(covex)))

In [30]:
# L7_pre = preprocessing(L7,Cloud_masking = True, calculateReflectance = True, CalculateVIs = False, Landsat = 7)

collectionList = L7.toList(L7.size())
collectionSize = collectionList.size().getInfo()
collectionSize
# ee.Image(L7_pre.toList(collectionSize).get(1)).projection().nominalScale().multiply(10).getInfo()

4

In [31]:
def ee_export_image(ee_object, filename, scale=None, crs=None, region=None, file_per_band=False):
    """Exports an ee.Image as a GeoTIFF.

    Args:
        ee_object (object): The ee.Image to download.
        filename (str): Output filename for the exported image.
        scale (float, optional): A default scale to use for any bands that do not specify one; ignored if crs and crs_transform is specified. Defaults to None.
        crs (str, optional): A default CRS string to use for any bands that do not explicitly specify one. Defaults to None.
        region (object, optional): A polygon specifying a region to download; ignored if crs and crs_transform is specified. Defaults to None.
        file_per_band (bool, optional): Whether to produce a different GeoTIFF per band. Defaults to False.
    """
    import requests
    import zipfile
#     ee_initialize()

    if not isinstance(ee_object, ee.Image):
        print('The ee_object must be an ee.Image.')
        return

    filename = os.path.abspath(filename)
    basename = os.path.basename(filename)
    name = os.path.splitext(basename)[0]
    filetype = os.path.splitext(basename)[1][1:].lower()
    filename_zip = filename.replace('.tif', '.zip')

    if filetype != 'tif':
        print('The filename must end with .tif')
        return

    try:
        print('Generating URL ...')
        params = {'name': name, 'filePerBand': file_per_band,'maxPixels': 1e13}
        if scale is None:
            scale = ee_object.projection().nominalScale().multiply(10)
        params['scale'] = scale
        if region is None:
            region = ee_object.geometry()
        params['region'] = region
        if crs is not None:
            params['crs'] = crs
        print(params)
        url = ee_object.getDownloadURL(params)
        print('Downloading data from {}\nPlease wait ...'.format(url))
        
        r = requests.get(url, stream=True)

        if r.status_code != 200:
            print('An error occurred while downloading.')
            return

        with open(filename_zip, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=1024):
                fd.write(chunk)

    except Exception as e:
        print('An error occurred while downloading.')
        print(e)
        return

    try:
        z = zipfile.ZipFile(filename_zip)
        z.extractall(os.path.dirname(filename))
        os.remove(filename_zip)

        if file_per_band:
            print('Data downloaded to {}'.format(os.path.dirname(filename)))
        else:
            print('Data downloaded to {}'.format(filename))
    except Exception as e:
        print(e)



def ee_export_image_collection(ee_object, out_dir, scale=None, crs=None, region=None, file_per_band=False):
    """Exports an ImageCollection as GeoTIFFs.

    Args:
        ee_object (object): The ee.Image to download.
        out_dir (str): The output directory for the exported images.
        scale (float, optional): A default scale to use for any bands that do not specify one; ignored if crs and crs_transform is specified. Defaults to None.
        crs (str, optional): A default CRS string to use for any bands that do not explicitly specify one. Defaults to None.
        region (object, optional): A polygon specifying a region to download; ignored if crs and crs_transform is specified. Defaults to None.
        file_per_band (bool, optional): Whether to produce a different GeoTIFF per band. Defaults to False.
    """

    import requests
    import zipfile
#     ee_initialize()

    if not isinstance(ee_object, ee.ImageCollection):
        print('The ee_object must be an ee.ImageCollection.')
        return

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    try:

        count = int(ee_object.size().getInfo())
        print("Total number of images: {}\n".format(count))

        for i in range(0, count):
            image = ee.Image(ee_object.toList(count).get(i))
            name = image.get('system:index').getInfo() + '.tif'
            filename = os.path.join(os.path.abspath(out_dir), name)
            print('Exporting {}/{}: {}'.format(i+1, count, name))
            ee_export_image(image, filename=filename, scale=scale,
                            crs=crs, region=region, file_per_band=file_per_band)
            print('\n')

    except Exception as e:
        print(e)


In [33]:
ee_export_image_collection(L7,"IMG",scale = 30)
# 

Total number of images: 4

Exporting 1/4: LE07_008046_20110117.tif
Generating URL ...
{'name': 'LE07_008046_20110117', 'filePerBand': False, 'maxPixels': 10000000000000.0, 'scale': 30, 'region': <ee.geometry.Geometry object at 0x0000013DF5762910>}
An error occurred while downloading.
Total request size (358647060 bytes) must be less than or equal to 33554432 bytes.


Exporting 2/4: LE07_008047_20110101.tif
Generating URL ...
{'name': 'LE07_008047_20110101', 'filePerBand': False, 'maxPixels': 10000000000000.0, 'scale': 30, 'region': <ee.geometry.Geometry object at 0x0000013DF5762910>}
An error occurred while downloading.
Total request size (358647060 bytes) must be less than or equal to 33554432 bytes.


Exporting 3/4: LE07_008047_20110117.tif
Generating URL ...
{'name': 'LE07_008047_20110117', 'filePerBand': False, 'maxPixels': 10000000000000.0, 'scale': 30, 'region': <ee.geometry.Geometry object at 0x0000013DF5762FA0>}
An error occurred while downloading.
Total request size (358647060